In [1]:
%cd ~/protein-transfer

/home/francesca/protein-transfer


In [2]:
%load_ext blackcellmagic

In [3]:
from scr.utils import pickle_load
from scr.analysis.perlayer import LayerLoss

In [4]:
LayerLoss(
    input_path="results/pytorch-carp",
    output_path="results/pytorch-carp_layer"
)

Making results/pytorch-carp_layer/structure/secondary_structure/tape_ss3_processed/carp_640M ...
Making results/pytorch-carp_layer/structure/secondary_structure/tape_ss3_processed/carp_640M/noflatten ...
Making results/pytorch-carp_layer-0.5/structure/secondary_structure/tape_ss3_processed/carp_640M ...
Making results/pytorch-carp_layer-0.5/structure/secondary_structure/tape_ss3_processed/carp_640M/noflatten ...
Making results/pytorch-carp_layer-0.25/structure/secondary_structure/tape_ss3_processed/carp_640M ...
Making results/pytorch-carp_layer-0.25/structure/secondary_structure/tape_ss3_processed/carp_640M/noflatten ...
Making results/pytorch-carp_layer-0.125/structure/secondary_structure/tape_ss3_processed/carp_640M ...
Making results/pytorch-carp_layer-0.125/structure/secondary_structure/tape_ss3_processed/carp_640M/noflatten ...
Making results/pytorch-carp_layer-rand/structure/secondary_structure/tape_ss3_processed/carp_640M ...
Making results/pytorch-carp_layer-rand/structure/sec

In [4]:
LayerLoss(
    input_path="results/pytorch-esm",
    output_path="results/pytorch-esm_layer",
    add_checkpoint=False
)

Plotting collage_name structure_secondary_structure_tape_ss3_processed_noflatten...
Plotting collage_name annotation_scl_balanced_mean...


In [5]:
carp_layerloss = LayerLoss(
    input_path="results/sklearn-carp",
    output_path="results/sklearn-carp_layer"
)

Plotting collage_name proeng_aav_one_vs_many_mean...
Plotting collage_name proeng_aav_two_vs_many_mean...
Plotting collage_name proeng_gb1_low_vs_high_mean...
Plotting collage_name proeng_gb1_two_vs_rest_mean...
Plotting collage_name proeng_gb1_sampled_mean...
Plotting collage_name proeng_thermo_mixed_split_mean...


In [ ]:
LayerLoss(
    input_path="test/sklearn",
    output_path="test/sklearn_layer")

In [11]:
from scr.utils import pickle_load

In [15]:
gb1_sample_output_dict = pickle_load("results/sklearn-scaley-noloader-fixembpool/proeng/gb1/sampled/esm1_t6_43M_UR50S/mean/esm1_t6_43M_UR50S-mean-layer_0.pkl")
gb1_sample_output_dict.keys(), gb1_sample_output_dict["test"].keys()

(dict_keys(['train', 'val', 'test']),
 dict_keys(['mse', 'pred', 'true', 'ndcg', 'rho']))

In [16]:
from scipy.stats import spearmanr

In [18]:
gb1_sample_output_dict["test"]["true"]

array([1.69016355e+00, 5.85648854e-01, 3.05454865e+00, ...,
       2.25273974e-03, 2.62819636e-02, 4.49316942e-03])

In [19]:
gb1_sample_output_dict["test"]["pred"]

array([4.51927324e-17, 4.51927324e-17, 4.51927324e-17, ...,
       4.51927324e-17, 4.51927324e-17, 4.51927324e-17])

In [17]:
spearmanr(gb1_sample_output_dict["test"]["true"], gb1_sample_output_dict["test"]["pred"])

/anaconda/envs/protran/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


SpearmanrResult(correlation=nan, pvalue=nan)